Copyright 2021 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


In [ ]:
#@title Initialization
reset_package = False #@param {type:"boolean"}
package_url = "https://github.com/google-research/google-research/trunk/m_theory" #@param {type:"string"}
package_source = "Fetch from Repository" #@param ["Upload from Computer", "Fetch from Repository"]

import glob
import os
import shlex
import shutil
import subprocess
import time


def _get_archive_via_upload(filename='m_theory.zip'):
  from google.colab import files
  while True:
    print('Please upload', filename)
    uploaded = files.upload()
    print('Uploaded:', {k: len(v) for k, v in uploaded.items()})
    if filename in uploaded:
      break
  subprocess.call(['unzip', filename])


def _get_archive_from_repository(url=None):
  import os
  import shlex
  if url is None:
    url = package_url
  os.system('[ -f /usr/bin/svn ] || apt install subversion')  # Subshell!
  subprocess.run(['/usr/bin/svn', 'export', url])

###
for filename in glob.glob('*.zip'):
  print('Removing:', filename)
  os.unlink(filename)

if reset_package:
  try:
    shutil.rmtree('m_theory')
  except OSError:
    pass

while not os.access('m_theory', os.X_OK):
  time.sleep(0.5)  # If something goes wrong here, do not fast-loop-on-failure.
  print('Obtaining package...')
  if package_source.startswith('Upload '):
    _get_archive_via_upload()
  elif package_source.startswith('Fetch '):
    _get_archive_from_repository(package_url)
  else:
    print('Mangled package source, aborting.')
    break

if os.access('m_theory', os.X_OK):
  print('Archive is available.')
  # Symlink things upwards into the current directory.
  for filename in sorted(os.listdir('m_theory')):
    try:
      os.symlink(os.path.join('m_theory', filename), filename)
    except OSError:
      pass

In [ ]:
import numpy
import tensorflow as tf
from m_theory_lib import m_util as mu
from m_theory_lib import algebra
from m_theory_lib import supergravity
from dim4.so8.src import dyonic


# Demo: Scan some critical points of SO(8)c supergravity for omega=pi/8.
# The SL(2)x7 trajectory code is at: dim4/papers/bfis2021/analyze_sl2x7.py
sugra = dyonic.SO8c_SUGRA()
t_omega=mu.tff64(numpy.pi*0.125)

for nn, (pot, stat, pos) in zip(range(10), sugra.scan(t_omega=t_omega)):
  if stat > 1e-8:
    continue
  phys = sugra.get_physics(pos, {}, t_omega=t_omega)
  print(sugra.show_physics_text(phys))
